# Model Pruning

**Module 03 | Notebook 3 of 5**

Pruning removes unnecessary weights from neural networks, making them smaller and faster.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand different pruning strategies
2. Apply L1 unstructured pruning with PyTorch
3. Visualize sparsity in neural networks
4. Analyze the accuracy vs. sparsity trade-off

---

In [ ]:
%%capture
!pip install transformers torch matplotlib seaborn pandas

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

---

## What is Pruning?

### The Intuition

Neural networks are often **over-parameterized**. Many weights contribute little to the output.

```
BEFORE PRUNING:                AFTER PRUNING (50%):
●━━━━━━●━━━━━━●                ●━━━━━━●━━━━━━●
 \    /|\    /                  \      |\    /
  \  / | \  /                    \     | \  /
   \/  |  \/                      \    |  \/
   /\  |  /\                       \   |  /\
  /  \ | /  \                       \  | /  \
 /    \|/    \                       \ |/    \
●━━━━━━●━━━━━━●                ●      ●      ●
 \    /|\    /                        |\    /
  \  / | \  /                         | \  /
   \/  |  \/                          |  \/
   /\  |  /\                          |  /\
  /  \ | /  \                         | /  \
 /    \|/    \                        |/    \
●━━━━━━●━━━━━━●                ●      ●      ●

All connections             Weak connections removed
```

### Types of Pruning

| Type | Description | Pros | Cons |
|------|-------------|------|------|
| **Unstructured** | Remove individual weights | Most flexible, highest sparsity | Irregular memory access |
| **Structured** | Remove entire neurons/channels | Hardware-friendly | Less flexible |
| **Magnitude-based** | Remove smallest weights | Simple, effective | May miss important small weights |
| **Gradient-based** | Remove by gradient importance | More accurate | Requires training data |

---

## Load and Analyze the Model

In [ ]:
# Load a pre-trained model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Count parameters
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    nonzero = sum((p != 0).sum().item() for p in model.parameters())
    return total, nonzero

total, nonzero = count_parameters(model)
print(f"Model: {model_name}")
print(f"Total parameters: {total:,}")
print(f"Non-zero parameters: {nonzero:,}")
print(f"Sparsity: {(1 - nonzero/total)*100:.2f}%")

In [ ]:
# Visualize weight distribution before pruning
def get_all_weights(model):
    weights = []
    for name, param in model.named_parameters():
        if 'weight' in name and param.dim() >= 2:
            weights.extend(param.detach().cpu().flatten().numpy())
    return np.array(weights)

weights_before = get_all_weights(model)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(weights_before, bins=100, alpha=0.7, color='steelblue', edgecolor='black')
plt.xlabel('Weight Value')
plt.ylabel('Count')
plt.title('Weight Distribution (Before Pruning)')
plt.axvline(x=0, color='red', linestyle='--', label='Zero')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(np.abs(weights_before), bins=100, alpha=0.7, color='steelblue', edgecolor='black')
plt.xlabel('Absolute Weight Value')
plt.ylabel('Count')
plt.title('Absolute Weight Distribution')
plt.axvline(x=np.percentile(np.abs(weights_before), 20), color='red', linestyle='--', label='20% threshold')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nWeight statistics:")
print(f"  Min: {weights_before.min():.4f}")
print(f"  Max: {weights_before.max():.4f}")
print(f"  Mean: {weights_before.mean():.4f}")
print(f"  Std: {weights_before.std():.4f}")

---

## Apply Unstructured Pruning

In [ ]:
def apply_pruning(model, amount=0.3, prune_type='l1'):
    """
    Apply pruning to all Linear layers in the model.
    
    Args:
        model: PyTorch model
        amount: Fraction of parameters to prune (0-1)
        prune_type: 'l1' for magnitude-based, 'random' for random
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            if prune_type == 'l1':
                prune.l1_unstructured(module, name='weight', amount=amount)
            elif prune_type == 'random':
                prune.random_unstructured(module, name='weight', amount=amount)
    
    return model

def remove_pruning_reparametrization(model):
    """
    Make pruning permanent by removing the reparametrization.
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            try:
                prune.remove(module, 'weight')
            except:
                pass
    return model

In [ ]:
# Create a copy for pruning
model_pruned = deepcopy(model)

# Apply 30% pruning
prune_amount = 0.3
model_pruned = apply_pruning(model_pruned, amount=prune_amount)

print(f"Applied {prune_amount*100:.0f}% L1 unstructured pruning")

In [ ]:
# Check sparsity after pruning
total, nonzero = count_parameters(model_pruned)
sparsity = (1 - nonzero/total) * 100

print(f"After pruning:")
print(f"  Total parameters: {total:,}")
print(f"  Non-zero parameters: {nonzero:,}")
print(f"  Sparsity: {sparsity:.2f}%")

In [ ]:
# Visualize weight distribution after pruning
weights_after = get_all_weights(model_pruned)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Before
axes[0].hist(weights_before, bins=100, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Weight Value')
axes[0].set_ylabel('Count')
axes[0].set_title('Before Pruning')

# After
axes[1].hist(weights_after, bins=100, alpha=0.7, color='coral', edgecolor='black')
axes[1].set_xlabel('Weight Value')
axes[1].set_ylabel('Count')
axes[1].set_title(f'After Pruning ({prune_amount*100:.0f}%)')

# Add zero spike annotation
zero_count = (np.abs(weights_after) < 1e-8).sum()
axes[1].annotate(f'{zero_count:,} zeros\n({zero_count/len(weights_after)*100:.1f}%)', 
                 xy=(0, axes[1].get_ylim()[1]*0.5),
                 fontsize=10, ha='center')

plt.tight_layout()
plt.show()

---

## Visualize Sparsity per Layer

In [ ]:
def get_layer_sparsity(model):
    """Calculate sparsity for each layer."""
    sparsity_data = []
    
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            weight = module.weight.data
            total = weight.numel()
            zeros = (weight == 0).sum().item()
            sparsity = zeros / total * 100
            
            sparsity_data.append({
                'Layer': name.split('.')[-2] + '.' + name.split('.')[-1] if '.' in name else name,
                'Total Params': total,
                'Zeros': zeros,
                'Sparsity (%)': sparsity
            })
    
    return pd.DataFrame(sparsity_data)

sparsity_df = get_layer_sparsity(model_pruned)
print("Layer-wise Sparsity:")
print(sparsity_df.to_string(index=False))

In [ ]:
# Visualize sparsity per layer
plt.figure(figsize=(12, 6))
plt.barh(range(len(sparsity_df)), sparsity_df['Sparsity (%)'], color='coral')
plt.yticks(range(len(sparsity_df)), sparsity_df['Layer'], fontsize=8)
plt.xlabel('Sparsity (%)')
plt.title('Sparsity per Layer')
plt.axvline(x=30, color='red', linestyle='--', label='Target (30%)')
plt.legend()
plt.tight_layout()
plt.show()

---

## Evaluate Accuracy Impact

In [ ]:
# Load test data
dataset = load_dataset("glue", "sst2")
test_data = dataset['validation'].shuffle(seed=42).select(range(500))

def evaluate_accuracy(model, tokenizer, test_data, device='cpu'):
    model = model.to(device)
    model.eval()
    correct = 0
    
    with torch.no_grad():
        for example in test_data:
            inputs = tokenizer(
                example['sentence'],
                return_tensors='pt',
                truncation=True,
                max_length=128
            ).to(device)
            
            outputs = model(**inputs)
            pred = outputs.logits.argmax(dim=-1).item()
            
            if pred == example['label']:
                correct += 1
    
    return correct / len(test_data)

print("Evaluating models...")

In [ ]:
# Compare accuracy
acc_original = evaluate_accuracy(model, tokenizer, test_data, device)
acc_pruned = evaluate_accuracy(model_pruned, tokenizer, test_data, device)

print(f"\nAccuracy Comparison:")
print(f"  Original Model: {acc_original:.2%}")
print(f"  Pruned Model ({prune_amount*100:.0f}%): {acc_pruned:.2%}")
print(f"  Accuracy Drop: {(acc_original - acc_pruned)*100:.2f}%")

---

## Sparsity vs. Accuracy Trade-off

In [ ]:
# Test different pruning levels
prune_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
results = []

print("Testing different pruning levels...")
for level in prune_levels:
    # Create fresh copy
    test_model = deepcopy(model)
    
    if level > 0:
        test_model = apply_pruning(test_model, amount=level)
    
    # Evaluate
    acc = evaluate_accuracy(test_model, tokenizer, test_data, device)
    total, nonzero = count_parameters(test_model)
    actual_sparsity = (1 - nonzero/total) * 100
    
    results.append({
        'Target Sparsity': level * 100,
        'Actual Sparsity': actual_sparsity,
        'Accuracy': acc * 100
    })
    
    print(f"  {level*100:.0f}% pruning → {acc:.2%} accuracy")
    
    del test_model

results_df = pd.DataFrame(results)

In [ ]:
# Plot trade-off
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(results_df['Target Sparsity'], results_df['Accuracy'], 'bo-', linewidth=2, markersize=10)

# Highlight "sweet spot"
ax.axhspan(results_df['Accuracy'].iloc[0] - 2, results_df['Accuracy'].iloc[0], 
           alpha=0.2, color='green', label='<2% accuracy drop zone')

ax.set_xlabel('Sparsity (%)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Sparsity vs. Accuracy Trade-off', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()

# Annotate points
for _, row in results_df.iterrows():
    ax.annotate(f"{row['Accuracy']:.1f}%", 
                (row['Target Sparsity'], row['Accuracy']),
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nResults Summary:")
print(results_df.to_string(index=False))

---

## Making Pruning Permanent

In [ ]:
# Remove pruning reparametrization to make it permanent
model_final = deepcopy(model_pruned)
model_final = remove_pruning_reparametrization(model_final)

# Verify
total, nonzero = count_parameters(model_final)
print(f"Final pruned model:")
print(f"  Total parameters: {total:,}")
print(f"  Non-zero parameters: {nonzero:,}")
print(f"  Sparsity: {(1 - nonzero/total)*100:.2f}%")

In [ ]:
# Save the pruned model
save_path = "./pruned_model"
model_final.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nPruned model saved to {save_path}")

---

## 🎯 Student Challenge

### Challenge: Find Optimal Pruning Level

In [ ]:
# TODO: Find the maximum pruning level that keeps accuracy drop < 1%

# 1. Based on the results above, what is the sweet spot?
# 2. Try finer-grained pruning levels (e.g., 0.35, 0.40, 0.45)
# 3. What if you use random pruning instead of L1? Is L1 better?

# Your solution:


---

## Key Takeaways

1. **Pruning removes weights** based on importance (usually magnitude)
2. **Unstructured pruning** is flexible but may not speed up inference
3. **L1 pruning** removes smallest absolute values
4. **30-50% sparsity** often has minimal accuracy impact
5. **Trade-off exists** between sparsity and accuracy

---

## Next Steps

Continue to `04_quantization.ipynb` for precision reduction techniques!